In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geodatasets
import geopandas as gpd
import seaborn as sns
import shapely
from shapely.geometry import MultiPoint, Point, box
from sklearn.cluster import KMeans

In [ ]:
data_path = Path("../data/update version with CI.csv")

In [ ]:
raw = pd.read_csv(data_path)
raw.sample(5)

In [ ]:
geodatasets.data.geoda.seattle1

In [ ]:
import requests
print(requests.certs.where())

had to manually copy the TMO cert into that cacert.pem file.

In [ ]:
seattle = gpd.read_file(geodatasets.get_path("geoda.seattle1"))

In [ ]:
seattle.crs

In [ ]:
seattle.plot()

In [ ]:
raw.head().transpose()

In [ ]:
raw.info()

In [ ]:
raw[raw['ED_ENVIRONMENT_LOCATION_TIMESTAMP'].isna()].transpose()

In [ ]:
def hour_to_period(hr: int) -> str:
    """convert a 24-hour number to one of a set of periods using a piecewise fn"""
    if hr <= 5 or hr >= 22:
        return "night"
    elif 18 > hr > 5:
        return "day"
    else:
        return "evening"

In [ ]:
geo = raw.dropna(subset=["ED_ENVIRONMENT_LOCATION_TIMESTAMP", "SITE_LONGITUDE", "SITE_LATITUDE"]).assign(
    device=raw['EVENT_DEVICE_DATA_DEVICEIMEI'].str[:5],
    time = (pd.to_datetime(raw.ED_ENVIRONMENT_LOCATION_TIMESTAMP.dropna()).dt.hour+1).apply(hour_to_period)
)

In [ ]:
gdf = gpd.GeoDataFrame(geo, geometry=geo[['SITE_LONGITUDE','SITE_LATITUDE']].apply(Point, axis=1)).set_crs(epsg=4326)

In [ ]:
ax = seattle.plot()
gdf.plot(ax=ax, color='red', marker='.', alpha=0.1)

Did one of these guys go in the Sound? Something's off here

In [ ]:
g = sns.FacetGrid(gdf,col="device", hue='time', col_wrap=3)
g.map(sns.scatterplot,"SITE_LONGITUDE", "SITE_LATITUDE", alpha=0.05) 
g.set_titles("{col_name}")
g.add_legend()

In [ ]:
g = sns.FacetGrid(gdf[gdf.ED_ENVIRONMENT_LOCATION_SPEED < 1],col="device", hue='time', col_wrap=3)
g.map(sns.scatterplot,"SITE_LONGITUDE", "SITE_LATITUDE", alpha=0.05) 
g.set_titles("{col_name}")
g.add_legend()
g.figure.suptitle("Stationary only")
g.figure.subplots_adjust(top=0.9)

# Clustering

We need to figure out a heuristic for n_neighbors. Since we have lat-lons this needn't be arbitrary, but we could just gridsearch.

In [ ]:
imei = gdf.EVENT_DEVICE_DATA_DEVICEIMEI.sample(1).iloc[0]
imei

In [ ]:
plt.plot(range(200), np.emath.logn(10, np.array(range(200))))

In [ ]:
box(*gdf[gdf.EVENT_DEVICE_DATA_DEVICEIMEI == imei].total_bounds).area

In [ ]:
def n_cluster_heuristic(grp) -> int:
    """determine how many clusters we should use on a given group:

    log base 10 of total KM**2 area of person's bounding box
    """
    return max(1,round(np.emath.logn(10, box(*grp.total_bounds).area * 111**2)))
    
    
def make_clusters(group: gpd.GeoDataFrame, n_clusters: int = None):
    if n_clusters is None:
        n_clusters = n_cluster_heuristic(group)
    lon_lat = group[["SITE_LONGITUDE","SITE_LATITUDE"]]
    kms = KMeans(n_clusters=n_clusters)
    kms.fit(lon_lat)
    group['cluster'] = kms.labels_
    return group

In [ ]:
clustered_geo = gdf.groupby("EVENT_DEVICE_DATA_DEVICEIMEI").apply(make_clusters, include_groups=False)

In [ ]:
clustered_geo.head()

In [ ]:
g = sns.FacetGrid(clustered_geo.reset_index(),col="device", hue='cluster', col_wrap=4)
g.map(sns.scatterplot,"SITE_LONGITUDE", "SITE_LATITUDE", alpha=0.05) 
g.set_titles("{col_name}")
g.add_legend()
g.figure.suptitle("Variable clusters")
g.figure.subplots_adjust(top=0.9)

May want to adjust the heuristic

# Generate Geofences

This should be pretty straightforward using Shapely

In [ ]:
clustered_geo.columns

In [ ]:
type(MultiPoint(clustered_geo.sample(10)['geometry'].values).buffer(distance=1/111., quad_segs=5))

In [ ]:
shapely.buffer(clustered_geo.sample(10)['geometry'].values, distance=1.)

In [ ]:
# Create a convex hull from the points
convex_hull = MultiPoint(clustered_geo.sample(45)['geometry'].values).convex_hull

# Simplify the polygon to reduce the number of sides
# The tolerance parameter controls the degree of simplification
convex_hull.simplify(tolerance=0.1, preserve_topology=True)

In [ ]:
convex_hull

In [ ]:
convex_hull.simplify(tolerance=1, preserve_topology=True)

Need to convert MultiPoint to a polygon. We do this with `.convex_hull`. We could then call `.simplify` to further reduce the polygon, but from experimentation I don't think it's necessary.

In [ ]:
geofences = (
    clustered_geo
    .reset_index()  # can't mix level and by in a groupby
    .groupby(by=['EVENT_DEVICE_DATA_DEVICEIMEI', "cluster"])
    .apply(lambda grp: MultiPoint(grp['geometry'].values).convex_hull, include_groups=False)
)

In [ ]:
geofences

In [ ]:
geofences.loc[imei].reset_index().apply(lambda x: plt.plot(*x.iloc[1].exterior.xy, c=f"C{x.iloc[0]}", label=x.iloc[0]),axis=1)
plt.legend(title="cluster")

Let's look at a more complex one

In [ ]:
geofences.loc[geofences.index[-1][0]].reset_index().apply(lambda x: plt.plot(*x.iloc[1].exterior.xy, c=f"C{x.iloc[0]}", label=x.iloc[0]),axis=1)
plt.legend(title="cluster")